In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import sys
!{sys.executable} -m pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import sys
!{sys.executable} -m pip install folium
!{sys.executable} -m pip install geocoder
import geocoder
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#import sys
#!{sys.executable} -m pip install geopandas
#!{sys.executable} -m pip install geopy
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

print('Libraries imported.')

     |████████████████████████████████| 102kB 9.3MB/s ta 0:00:011
     |████████████████████████████████| 102kB 9.9MB/s eta 0:00:01
Libraries imported.


In [2]:
# The code was removed by Watson Studio for sharing.

,Postcode,Borough,Neighborhood
0,01312010,Bela Vista,Praça Quatorze Bis - lado par
1,03043900,Brás,"Rua Alegria, 300"
2,03041010,Brás,Rua Paraná
3,01310919,Bela Vista,"Avenida Paulista, 1728"
4,01309030,Consolação,Rua Fernando de Albuquerque


In [3]:
df = df_new.copy()

locator = Nominatim(user_agent='myGeocoder')

def get_coordinates(arcgis_geocoder):
    
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Sao Paulo, Brazil'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    
    return lat_lng_coords

postcodes = df['Postcode']  

coordinates = [get_coordinates(code) for code in postcodes.tolist()]
df_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])

df['Latitude'] = df_coordinates['Latitude']
df['Longitude'] = df_coordinates['Longitude']

# Get names of indexes for which column Age has value 30
indexNames = df[np.isnan(df['Latitude']) ].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,01312010,Bela Vista,Praça Quatorze Bis - lado par,-23.555532,-46.650080
1,03043900,Brás,"Rua Alegria, 300",-23.549100,-46.615549
2,03041010,Brás,Rua Paraná,-23.550919,-46.618377
3,01310919,Bela Vista,"Avenida Paulista, 1728",-23.559007,-46.658125
4,01309030,Consolação,Rua Fernando de Albuquerque,-23.553803,-46.657978


In [4]:
CLIENT_ID = 'IZJGJKN3M0C1T5O2RYVADOE3EFJWTQUYUBHVWU02DS20VBOF' # your Foursquare ID
CLIENT_SECRET = 'BN1IPXLCWPY4XL5OAQYYTM4NNZPQ0R114AHPT5ZDTTKGE3NV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        #print(url)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

    return(nearby_venues)

LIMIT = 100
venues = getNearbyVenues(names = df['Postcode'], latitudes=df['Latitude'], longitudes=df['Longitude'])

venues.head()

,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,01312010,-23.555532,-46.650080,Frutaria E Cafeteria Bella Paim,-23.554799,-46.650001,Coffee Shop
1,01312010,-23.555532,-46.650080,Bar Brahma,-23.555664,-46.649738,Bar
2,01312010,-23.555532,-46.650080,Lanchonete Top Bis,-23.555549,-46.650022,Brazilian Restaurant
3,03043900,-23.549100,-46.615549,Restaurante Allegro,-23.549269,-46.615348,Brazilian Restaurant
4,03043900,-23.549100,-46.615549,Ponto Terminal Grajaú 5630-10,-23.548449,-46.616106,Bus Station


In [6]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Postcode'] = venues['Postcode'] 


# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

grouped = onehot.groupby('Postcode').mean().reset_index()

grouped.head()

,Postcode,Accessories Store,Art Gallery,Arts & Crafts Store,Bakery,Bar,Bookstore,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Bus Station,Café,Chocolate Shop,Clothing Store,Coffee Shop,College Bookstore,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Empada House,Farmers Market,Fast Food Restaurant,Food & Drink Shop,French Restaurant,Gastropub,Gay Bar,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Historic Site,Hobby Shop,Hot Dog Joint,Hotel,Ice Cream Shop,Italian Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Nightclub,Office,Organic Grocery,Peruvian Restaurant,Pet Store,Photography Studio,Pizza Place,Plaza,Record Shop,Restaurant,Sandwich Place,Shop & Service,Spa,Sporting Goods Shop,Supermarket,Sushi Restaurant,Theater,Vegetarian / Vegan Restaurant,Warehouse Store,Wine Shop,Women's Store,Yoga Studio
0,01002010,0.000,0.125000,0.0,0.125000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.125000,0.0,0.0,0.125000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125000,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.000,0.125000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
1,01002903,0.000,0.142857,0.0,0.142857,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.142857,0.0,0.0,0.142857,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.000,0.142857,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
2,01003902,0.000,0.000000,0.0,0.166667,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.000,0.166667,0.0,0.0,0.000000,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.000,0.166667,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
3,01003905,0.000,0.000000,0.0,0.166667,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.000,0.166667,0.0,0.0,0.000000,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.000,0.166667,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
4,01043000,0.125,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.125,0.125000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.125,0.0,0.0,0.125,0.000000,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0


In [7]:
# set number of clusters
kclusters = 20

grouped_clustering = grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(init='k-means++', n_clusters = kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([ 1,  1,  1,  1,  8, 13,  9,  9, 12, 16], dtype=int32)

In [8]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Postcode'] = grouped['Postcode']

for ind in np.arange(grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,01002010,College Bookstore,Grocery Store,Chocolate Shop,Sporting Goods Shop,Gift Shop,Jewelry Store,Bakery,Art Gallery,Gay Bar,Gym
1,01002903,College Bookstore,Gift Shop,Chocolate Shop,Sporting Goods Shop,Jewelry Store,Bakery,Art Gallery,Grocery Store,Gay Bar,Gym
2,01003902,Cosmetics Shop,Bakery,Bookstore,Gift Shop,Sporting Goods Shop,Chocolate Shop,Food & Drink Shop,Dessert Shop,Empada House,Farmers Market
3,01003905,Cosmetics Shop,Bakery,Bookstore,Gift Shop,Sporting Goods Shop,Chocolate Shop,Food & Drink Shop,Dessert Shop,Empada House,Farmers Market
4,01043000,Record Shop,Accessories Store,Spa,Café,Restaurant,Chocolate Shop,Wine Shop,Bar,Cosmetics Shop,Grocery Store


In [9]:
# add clustering labels
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
df_cluster = df.copy()

df_cluster = df_cluster.join(venues_sorted.set_index('Postcode'), on='Postcode')

df_cluster.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,01312010,Bela Vista,Praça Quatorze Bis - lado par,-23.555532,-46.650080,0.0,Coffee Shop,Bar,Brazilian Restaurant,Yoga Studio,French Restaurant,Dessert Shop,Empada House,Farmers Market,Fast Food Restaurant,Food & Drink Shop
1,03043900,Brás,"Rua Alegria, 300",-23.549100,-46.615549,11.0,Bus Station,Brazilian Restaurant,Yoga Studio,French Restaurant,Dessert Shop,Empada House,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Gastropub
2,03041010,Brás,Rua Paraná,-23.550919,-46.618377,5.0,Middle Eastern Restaurant,Yoga Studio,Food & Drink Shop,Department Store,Dessert Shop,Empada House,Farmers Market,Fast Food Restaurant,French Restaurant,Historic Site
3,01310919,Bela Vista,"Avenida Paulista, 1728",-23.559007,-46.658125,10.0,Juice Bar,Dessert Shop,Gift Shop,Ice Cream Shop,Plaza,Brazilian Restaurant,Food & Drink Shop,Empada House,Farmers Market,Fast Food Restaurant
4,01309030,Consolação,Rua Fernando de Albuquerque,-23.553803,-46.657978,13.0,French Restaurant,Vegetarian / Vegan Restaurant,Bar,Café,Yoga Studio,Dessert Shop,Empada House,Farmers Market,Fast Food Restaurant,Food & Drink Shop


In [10]:
# Get names of indexes for which column Age has value 30
indexNames = df_cluster[np.isnan(df_cluster['Cluster Labels']) ].index
# Delete these row indexes from dataFrame
df_cluster.drop(indexNames , inplace=True)

In [11]:
df_cluster['Cluster Labels'] = df_cluster['Cluster Labels'].astype('int64')

g = geocoder.arcgis('Sao Paulo, Brazil')
lat_lng_coords = g.latlng

# create map
map_clusters = folium.Map(location=[lat_lng_coords[0], lat_lng_coords[1]], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_cluster['Latitude'], df_cluster['Longitude'], df_cluster['Neighborhood'], df_cluster['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [12]:
df_cluster.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,01312010,Bela Vista,Praça Quatorze Bis - lado par,-23.555532,-46.650080,0,Coffee Shop,Bar,Brazilian Restaurant,Yoga Studio,French Restaurant,Dessert Shop,Empada House,Farmers Market,Fast Food Restaurant,Food & Drink Shop
1,03043900,Brás,"Rua Alegria, 300",-23.549100,-46.615549,11,Bus Station,Brazilian Restaurant,Yoga Studio,French Restaurant,Dessert Shop,Empada House,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Gastropub
2,03041010,Brás,Rua Paraná,-23.550919,-46.618377,5,Middle Eastern Restaurant,Yoga Studio,Food & Drink Shop,Department Store,Dessert Shop,Empada House,Farmers Market,Fast Food Restaurant,French Restaurant,Historic Site
3,01310919,Bela Vista,"Avenida Paulista, 1728",-23.559007,-46.658125,10,Juice Bar,Dessert Shop,Gift Shop,Ice Cream Shop,Plaza,Brazilian Restaurant,Food & Drink Shop,Empada House,Farmers Market,Fast Food Restaurant
4,01309030,Consolação,Rua Fernando de Albuquerque,-23.553803,-46.657978,13,French Restaurant,Vegetarian / Vegan Restaurant,Bar,Café,Yoga Studio,Dessert Shop,Empada House,Farmers Market,Fast Food Restaurant,Food & Drink Shop
5,01310904,Bela Vista,"Avenida Paulista, 460",-23.559007,-46.658125,10,Juice Bar,Dessert Shop,Gift Shop,Ice Cream Shop,Plaza,Brazilian Restaurant,Food & Drink Shop,Empada House,Farmers Market,Fast Food Restaurant
6,01002010,Sé,Praça do Patriarca,-23.547434,-46.636345,1,College Bookstore,Grocery Store,Chocolate Shop,Sporting Goods Shop,Gift Shop,Jewelry Store,Bakery,Art Gallery,Gay Bar,Gym
7,01331904,Bela Vista,"Rua Doutor Seng, 152",-23.561202,-46.650955,6,Yoga Studio,Coffee Shop,Food & Drink Shop,Department Store,Dessert Shop,Empada House,Farmers Market,Fast Food Restaurant,French Restaurant,Cosmetics Shop
8,01002903,Sé,"Rua Direita, 250",-23.547617,-46.636470,1,College Bookstore,Gift Shop,Chocolate Shop,Sporting Goods Shop,Jewelry Store,Bakery,Art Gallery,Grocery Store,Gay Bar,Gym
9,01310905,Bela Vista,"Avenida Paulista, 352",-23.559007,-46.658125,10,Juice Bar,Dessert Shop,Gift Shop,Ice Cream Shop,Plaza,Brazilian Restaurant,Food & Drink Shop,Empada House,Farmers Market,Fast Food Restaurant


In [47]:
print('PLACES WITH PIZZA AS 1st COMMON VENUE:\n')
df_analysis = df_cluster[['Borough','Neighborhood','Cluster Labels','1st Most Common Venue']].copy()

clusters_pizza = []
places = []

for i in range(1,21):
    df_filter = df_analysis[df_analysis['Cluster Labels'] == i].copy()
    df_pizza = df_filter[df_filter['1st Most Common Venue'].str.contains('Pizza')]
    
    if df_pizza['1st Most Common Venue'].to_list():
        
        len_list = len(df_pizza['Borough'].to_list())
        for n in range(0,len_list):
            places.append(df_pizza['Neighborhood'].to_list()[n])
            print('You can open your Pizza on ' + df_pizza['Borough'].to_list()[n].upper() + ' around '+ df_pizza['Neighborhood'].to_list()[n].upper())
        
        print('Similar Places:')
        for index, row in df_filter.iterrows():
            if(row['Neighborhood'] not in places):
                print('   on ' + row['Borough'].upper() + ' around '+ row['Neighborhood'].upper())
                
        print('\n')    
        
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')  

#----------------------2nd MOST COMMON VENUE----------------------------------#
print('PLACES WITH PIZZA AS 2nd COMMON VENUE:\n')
df_analysis = df_cluster[['Borough','Neighborhood','Cluster Labels','2nd Most Common Venue']].copy()

for i in range(1,21):
    df_filter = df_analysis[df_analysis['Cluster Labels'] == i].copy()
    df_pizza = df_filter[df_filter['2nd Most Common Venue'].str.contains('Pizza')]
    
    if df_pizza['2nd Most Common Venue'].to_list():
        
        len_list = len(df_pizza['Borough'].to_list())
        for n in range(0,len_list):
            places.append(df_pizza['Neighborhood'].to_list()[n])
            print('You can open your Pizza on ' + df_pizza['Borough'].to_list()[n].upper() + ' around '+ df_pizza['Neighborhood'].to_list()[n].upper())
        
        print('Similar Places:')
        for index, row in df_filter.iterrows():
            if(row['Neighborhood'] not in places):
                print('   on ' + row['Borough'].upper() + ' around '+ row['Neighborhood'].upper())
                
        print('\n')      
        
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')  

#----------------------3rd MOST COMMON VENUE----------------------------------#
print('PLACES WITH PIZZA AS 3rd COMMON VENUE:\n')
df_analysis = df_cluster[['Borough','Neighborhood','Cluster Labels','3rd Most Common Venue']].copy()

for i in range(1,21):
    df_filter = df_analysis[df_analysis['Cluster Labels'] == i].copy()
    df_pizza = df_filter[df_filter['3rd Most Common Venue'].str.contains('Pizza')]
    
    if df_pizza['3rd Most Common Venue'].to_list():
        
        len_list = len(df_pizza['Borough'].to_list())
        for n in range(0,len_list):
            places.append(df_pizza['Neighborhood'].to_list()[n])
            print('You can open your Pizza on ' + df_pizza['Borough'].to_list()[n].upper() + ' around '+ df_pizza['Neighborhood'].to_list()[n].upper())
        
        print('Similar Places:')
        for index, row in df_filter.iterrows():
            if(row['Neighborhood'] not in places):
                print('   on ' + row['Borough'].upper() + ' around '+ row['Neighborhood'].upper())
                
        print('\n')
        
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')  

#----------------------4th MOST COMMON VENUE----------------------------------#
print('PLACES WITH PIZZA AS 4th COMMON VENUE:\n')
df_analysis = df_cluster[['Borough','Neighborhood','Cluster Labels','4th Most Common Venue']].copy()

for i in range(1,21):
    df_filter = df_analysis[df_analysis['Cluster Labels'] == i].copy()
    df_pizza = df_filter[df_filter['4th Most Common Venue'].str.contains('Pizza')]
    
    if df_pizza['4th Most Common Venue'].to_list():
        
        len_list = len(df_pizza['Borough'].to_list())
        for n in range(0,len_list):
            places.append(df_pizza['Neighborhood'].to_list()[n])
            print('You can open your Pizza on ' + df_pizza['Borough'].to_list()[n].upper() + ' around '+ df_pizza['Neighborhood'].to_list()[n].upper())
        
        print('Similar Places:')
        for index, row in df_filter.iterrows():
            if(row['Neighborhood'] not in places):
                print('   on ' + row['Borough'].upper() + ' around '+ row['Neighborhood'].upper())
                
        print('\n')
        
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')  


#----------------------5th MOST COMMON VENUE----------------------------------#
print('PLACES WITH PIZZA AS 5th COMMON VENUE:\n')
df_analysis = df_cluster[['Borough','Neighborhood','Cluster Labels','5th Most Common Venue']].copy()

for i in range(1,21):
    df_filter = df_analysis[df_analysis['Cluster Labels'] == i].copy()
    df_pizza = df_filter[df_filter['5th Most Common Venue'].str.contains('Pizza')]

    if df_pizza['5th Most Common Venue'].to_list():
        
        len_list = len(df_pizza['Borough'].to_list())
        for n in range(0,len_list):
            places.append(df_pizza['Neighborhood'].to_list()[n])
            print('You can open your Pizza on ' + df_pizza['Borough'].to_list()[n].upper() + ' around '+ df_pizza['Neighborhood'].to_list()[n].upper())
        
        print('Similar Places:')
        for index, row in df_filter.iterrows():
            if(row['Neighborhood'] not in places):
                print('   on ' + row['Borough'].upper() + ' around '+ row['Neighborhood'].upper())
                
        print('\n')

PLACES WITH PIZZA AS 1st COMMON VENUE:

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

PLACES WITH PIZZA AS 2nd COMMON VENUE:

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

PLACES WITH PIZZA AS 3rd COMMON VENUE:

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

PLACES WITH PIZZA AS 4th COMMON VENUE:

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

PLACES WITH PIZZA AS 5th COMMON VENUE:

You can open your Pizza on REPÚBLICA around AVENIDA VIEIRA DE CARVALHO, 115
You can open your Pizza on BELA VISTA around AVENIDA BRIGADEIRO LUÍS ANTÔNIO, 733
Similar Places:
   on REPÚBLICA around RUA REGO FREITAS, 354
   on LIBERDADE around RUA MAESTRO CARDIM - DE 561/562 AO FIM




In [44]:
print('PLACES WITH GYM AS 1st COMMON VENUE:\n')
df_analysis = df_cluster[['Borough','Neighborhood','Cluster Labels','1st Most Common Venue']].copy()

clusters_gym = []
places = []

for i in range(1,21):
    df_filter = df_analysis[df_analysis['Cluster Labels'] == i].copy()
    df_gym = df_filter[df_filter['1st Most Common Venue'].str.contains('Gym')]
    
    if df_gym['1st Most Common Venue'].to_list():
        
        len_list = len(df_gym['Borough'].to_list())
        for n in range(0,len_list):
            places.append(df_gym['Neighborhood'].to_list()[n])
            print('You can open your Gym on ' + df_gym['Borough'].to_list()[n].upper() + ' around '+ df_gym['Neighborhood'].to_list()[n].upper())
        
        print('Similar Places:')
        for index, row in df_filter.iterrows():
            if(row['Neighborhood'] not in places):
                print('   on ' + row['Borough'].upper() + ' around '+ row['Neighborhood'].upper())
                
        print('\n')    
        
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')  

#----------------------2nd MOST COMMON VENUE----------------------------------#
print('PLACES WITH GYM AS 2nd COMMON VENUE:\n')
df_analysis = df_cluster[['Borough','Neighborhood','Cluster Labels','2nd Most Common Venue']].copy()

for i in range(1,21):
    df_filter = df_analysis[df_analysis['Cluster Labels'] == i].copy()
    df_gym = df_filter[df_filter['2nd Most Common Venue'].str.contains('Gym')]
    
    if df_gym['2nd Most Common Venue'].to_list():
        
        len_list = len(df_gym['Borough'].to_list())
        for n in range(0,len_list):
            places.append(df_gym['Neighborhood'].to_list()[n])
            print('You can open your Gym on ' + df_gym['Borough'].to_list()[n].upper() + ' around '+ df_gym['Neighborhood'].to_list()[n].upper())
        
        print('Similar Places:')
        for index, row in df_filter.iterrows():
            if(row['Neighborhood'] not in places):
                print('   on ' + row['Borough'].upper() + ' around '+ row['Neighborhood'].upper())
                
        print('\n')      
        
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')  

#----------------------3rd MOST COMMON VENUE----------------------------------#
print('PLACES WITH GYM AS 3rd COMMON VENUE:\n')
df_analysis = df_cluster[['Borough','Neighborhood','Cluster Labels','3rd Most Common Venue']].copy()

for i in range(1,21):
    df_filter = df_analysis[df_analysis['Cluster Labels'] == i].copy()
    df_gym = df_filter[df_filter['3rd Most Common Venue'].str.contains('Gym')]
    
    if df_gym['3rd Most Common Venue'].to_list():
        
        len_list = len(df_gym['Borough'].to_list())
        for n in range(0,len_list):
            places.append(df_gym['Neighborhood'].to_list()[n])
            print('You can open your Gym on ' + df_gym['Borough'].to_list()[n].upper() + ' around '+ df_gym['Neighborhood'].to_list()[n].upper())
        
        print('Similar Places:')
        for index, row in df_filter.iterrows():
            if(row['Neighborhood'] not in places):
                print('   on ' + row['Borough'].upper() + ' around '+ row['Neighborhood'].upper())
                
        print('\n')
        
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')  

#----------------------4th MOST COMMON VENUE----------------------------------#
print('PLACES WITH GYM AS 4th COMMON VENUE:\n')
df_analysis = df_cluster[['Borough','Neighborhood','Cluster Labels','4th Most Common Venue']].copy()

for i in range(1,21):
    df_filter = df_analysis[df_analysis['Cluster Labels'] == i].copy()
    df_gym = df_filter[df_filter['4th Most Common Venue'].str.contains('Gym')]
    
    if df_gym['4th Most Common Venue'].to_list():
        
        len_list = len(df_gym['Borough'].to_list())
        for n in range(0,len_list):
            places.append(df_gym['Neighborhood'].to_list()[n])
            print('You can open your Gym on ' + df_gym['Borough'].to_list()[n].upper() + ' around '+ df_gym['Neighborhood'].to_list()[n].upper())
        
        print('Similar Places:')
        for index, row in df_filter.iterrows():
            if(row['Neighborhood'] not in places):
                print('   on ' + row['Borough'].upper() + ' around '+ row['Neighborhood'].upper())
                
        print('\n')
        
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')  


#----------------------5th MOST COMMON VENUE----------------------------------#
print('PLACES WITH GYM AS 5th COMMON VENUE:\n')
df_analysis = df_cluster[['Borough','Neighborhood','Cluster Labels','5th Most Common Venue']].copy()

for i in range(1,21):
    df_filter = df_analysis[df_analysis['Cluster Labels'] == i].copy()
    df_gym = df_filter[df_filter['5th Most Common Venue'].str.contains('Gym')]

    if df_gym['5th Most Common Venue'].to_list():
        
        len_list = len(df_gym['Borough'].to_list())
        for n in range(0,len_list):
            places.append(df_gym['Neighborhood'].to_list()[n])
            print('You can open your Gym on ' + df_gym['Borough'].to_list()[n].upper() + ' around '+ df_gym['Neighborhood'].to_list()[n].upper())
        
        print('Similar Places:')
        for index, row in df_filter.iterrows():
            if(row['Neighborhood'] not in places):
                print('   on ' + row['Borough'].upper() + ' around '+ row['Neighborhood'].upper())
                
        print('\n')

PLACES WITH GYM AS 1st COMMON VENUE:

You can open your Gym on SANTA CECÍLIA around RUA DOUTOR ALBUQUERQUE LINS, 574
Similar Places:


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

PLACES WITH GYM AS 2nd COMMON VENUE:

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

PLACES WITH GYM AS 3rd COMMON VENUE:

You can open your Gym on LIBERDADE around RUA MAESTRO CARDIM - DE 561/562 AO FIM
Similar Places:
   on REPÚBLICA around RUA REGO FREITAS, 354
   on REPÚBLICA around AVENIDA VIEIRA DE CARVALHO, 115
   on BELA VISTA around AVENIDA BRIGADEIRO LUÍS ANTÔNIO, 733


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

PLACES WITH GYM AS 4th COMMON VENUE:

You can open your Gym on BRÁS around RUA DOUTOR ALMEIDA LIMA - ATÉ 599/600
Similar Places:


You can open your Gym on BRÁS around PRAÇA DOMINGOS FRANGIONE
Similar Places:


You can open your Gym